In [42]:
from neo4j import GraphDatabase, basic_auth
import csv

In [32]:
patients_csv_data = []
with open("csv/patients.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        patients_csv_data.append(row)

In [43]:
patients_csv_data[0]

{'Id': 'b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85',
 'BIRTHDATE': '2019-02-17',
 'DEATHDATE': '',
 'SSN': '999-65-3251',
 'DRIVERS': '',
 'PASSPORT': '',
 'PREFIX': '',
 'FIRST': 'Damon455',
 'LAST': 'Langosh790',
 'SUFFIX': '',
 'MAIDEN': '',
 'MARITAL': '',
 'RACE': 'white',
 'ETHNICITY': 'nonhispanic',
 'GENDER': 'M',
 'BIRTHPLACE': 'Middleborough  Massachusetts  US',
 'ADDRESS': '620 Lynch Tunnel Apt 0',
 'CITY': 'Springfield',
 'STATE': 'Massachusetts',
 'COUNTY': 'Hampden County',
 'ZIP': '01104',
 'LAT': '42.08038942501558',
 'LON': '-72.48043144917739',
 'HEALTHCARE_EXPENSES': '9039.1645',
 'HEALTHCARE_COVERAGE': '7964.1255'}

In [41]:
def upload_records(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE(p:Patient {Id: record.Id})
    SET p += record
    """
    tx.run(cypher_query, records=records)

# Connect to Neo4j and upload data
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

with driver.session(database="neo4j") as session:
    session.execute_write(upload_records, patients_csv_data)

In [ ]:
def upload_conditions(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE(c:Conditions {Id: record.Id})
    SET p += record
    """
    tx.run(cypher_query, records=records)

# Connect to Neo4j and upload data
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

with driver.session(database="neo4j") as session:
    session.execute_write(upload_records, patients_csv_data)